In [1]:
import json
import numpy as np

from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform

In [2]:
parameter_grid = {
#     'epochs': [25, 50, 100, 200], # it doesn't make sense to search epochs, when we can use early stopping
    'batch_size': [2 ** n for n in range(5, 10 + 1)],
    'learning_rate': uniform(loc=2, scale=4 - 2), # N ~ uniform in the inverval [2; 4], then the learning rate will be 10 ^ (-N)
    'dropout_rate': uniform(loc=0.0, scale=0.2), # N ~ uniform in the interval [0.0; 0.2]
    'latent_dim': list(range(256, 1024 + 1, 128)),
    'embedding_dim': list(range(32, 128 + 1, 16)),
    'vocabulary_size': list(range(2000, 7000 + 1, 1000)),
    'max_input_seq_length': list(range(50, 200, 25)),
    'max_output_seq_length': list(range(5, 8 + 1)),
}
parameter_grid

{'batch_size': [32, 64, 128, 256, 512, 1024],
 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen at 0x7f2ef5770908>,
 'dropout_rate': <scipy.stats._distn_infrastructure.rv_frozen at 0x7f2ef5770e80>,
 'latent_dim': [256, 384, 512, 640, 768, 896, 1024],
 'embedding_dim': [32, 48, 64, 80, 96, 112, 128],
 'vocabulary_size': [2000, 3000, 4000, 5000, 6000, 7000],
 'max_input_seq_length': [50, 75, 100, 125, 150, 175],
 'max_output_seq_length': [5, 6, 7, 8]}

In [3]:
def round_float_values(obj):
    return {
        key: round(value, 6) if isinstance(value, float) else value
        for key, value in obj.items()
    }

In [4]:
def exponentiate_learning_rate(params):
    return {
        **params,
        'learning_rate': 10 ** -params['learning_rate']
    }

In [5]:
experiments = {
    f'experiment_{i}': round_float_values(exponentiate_learning_rate(params))
    for i, params in enumerate(ParameterSampler(
        parameter_grid,
        n_iter=20,
        random_state=0,
    ))
}

In [6]:
print(json.dumps(experiments, indent=2))

{
  "experiment_0": {
    "batch_size": 512,
    "dropout_rate": 0.118569,
    "embedding_dim": 32,
    "latent_dim": 640,
    "learning_rate": 0.000192,
    "max_input_seq_length": 75,
    "max_output_seq_length": 8,
    "vocabulary_size": 7000
  },
  "experiment_1": {
    "batch_size": 128,
    "dropout_rate": 0.087517,
    "embedding_dim": 128,
    "latent_dim": 256,
    "learning_rate": 0.000118,
    "max_input_seq_length": 100,
    "max_output_seq_length": 6,
    "vocabulary_size": 2000
  },
  "experiment_2": {
    "batch_size": 64,
    "dropout_rate": 0.185119,
    "embedding_dim": 112,
    "latent_dim": 256,
    "learning_rate": 0.006695,
    "max_input_seq_length": 125,
    "max_output_seq_length": 5,
    "vocabulary_size": 5000
  },
  "experiment_3": {
    "batch_size": 1024,
    "dropout_rate": 0.155631,
    "embedding_dim": 32,
    "latent_dim": 512,
    "learning_rate": 0.001129,
    "max_input_seq_length": 75,
    "max_output_seq_length": 8,
    "vocabulary_size": 7000
  }

In [7]:
with open('../experiments/random_search_1.json', 'w') as f:
    json.dump(experiments, f, indent=2)